# E91 Implementation

1. create bell pairs
2. choose measurement for alice, bob
    * rotate $|0\rangle$, $|1\rangle$ by $a \in \{0,\frac{\pi}{4},\frac{\pi}{2}\}$ or $b \in \{\frac{\pi}{4},\frac{\pi}{2},\frac{3\pi}{4}\}$

Source: [https://journals.aps.org/prl/pdf/10.1103/PhysRevLett.67.661](https://journals.aps.org/prl/pdf/10.1103/PhysRevLett.67.661)